In [1]:
# Q: how do i feed my model all the data points in the training set
# A: Mini-Batch SGD ?

# Q: What is the train set, dev set and test set ?

# Q: Read up about Model Smoothing / Regularization

# Q: Read up about Cross Entropy Loss

# Q: Initialising of weights

In [26]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline
NUM_CLASSES = 27*28
T = torch.tensor

g = torch.Generator().manual_seed(2147483647)

In [24]:
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
df1 = pd.read_csv(url)

In [25]:
words = ['emma']
for word in df1['emma']:
  words.append(word)

In [38]:
trainingDataSet = []
devDataSet = []
evalDataSet = []
ind = 0

p = T([0.8, 0.1, 0.1])
multiSamples = multiSamples = torch.multinomial(p, num_samples=len(words), replacement=True, generator=g)
# print(multiSamples)

for i in multiSamples:
    if i == 0: trainingDataSet.append(words[ind])
    elif i == 1: devDataSet.append(words[ind])
    else: evalDataSet.append(words[ind])
    ind += 1

0.09939749633190773


In [39]:
for word in words[:1]:
  word = '.' + word + '.'
  for ch1, ch2, ch3 in zip(word, word[1:], word[2:]):
    print(f"<{ch1}{ch2}> {ch3}")
    

<.e> m
<em> m
<mm> a
<ma> .


In [13]:
chars = ['.'] + list(map(chr, range(97, 123)))
doubleChars = []

for ch1 in chars:
  for ch2 in chars:
    doubleChars.append(ch1 + ch2)

totalChars = chars + doubleChars

756 ['.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '..', '.a', '.b', '.c', '.d', '.e', '.f', '.g', '.h', '.i', '.j', '.k', '.l', '.m', '.n', '.o', '.p', '.q', '.r', '.s', '.t', '.u', '.v', '.w', '.x', '.y', '.z', 'a.', 'aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai', 'aj', 'ak', 'al', 'am', 'an', 'ao', 'ap', 'aq', 'ar', 'as', 'at', 'au', 'av', 'aw', 'ax', 'ay', 'az', 'b.', 'ba', 'bb', 'bc', 'bd', 'be', 'bf', 'bg', 'bh', 'bi', 'bj', 'bk', 'bl', 'bm', 'bn', 'bo', 'bp', 'bq', 'br', 'bs', 'bt', 'bu', 'bv', 'bw', 'bx', 'by', 'bz', 'c.', 'ca', 'cb', 'cc', 'cd', 'ce', 'cf', 'cg', 'ch', 'ci', 'cj', 'ck', 'cl', 'cm', 'cn', 'co', 'cp', 'cq', 'cr', 'cs', 'ct', 'cu', 'cv', 'cw', 'cx', 'cy', 'cz', 'd.', 'da', 'db', 'dc', 'dd', 'de', 'df', 'dg', 'dh', 'di', 'dj', 'dk', 'dl', 'dm', 'dn', 'do', 'dp', 'dq', 'dr', 'ds', 'dt', 'du', 'dv', 'dw', 'dx', 'dy', 'dz', 'e.', 'ea', 'eb', 'ec', 'ed', 'ee', 'ef', 'eg', 'e

In [14]:
stoi = {chr: i for i, chr in enumerate(totalChars)} 

itos = {i: chr for chr, i in stoi.items()}

In [16]:
inputs = []
labels = []
trainingSetSize = 0
for word in trainingDataSet:  
  word = '.' + word + '.'
  for ch1, ch2, ch3 in zip(word, word[1:], word[2:]):
    # print(f"<{ch1}{ch2}> {ch3}")

    y = stoi[ch3]
    x = stoi[ch1 + ch2] 

    inputs.append(x)
    labels.append(y)
    trainingSetSize += 1
    
inputs = T(inputs)
labels = T(labels)
print(inputs, len(inputs))
print(labels, len(labels))

tensor([ 32, 175, 391,  ..., 360, 271,  62]) 6000
tensor([13, 13,  1,  ...,  1,  8,  0]) 6000


In [110]:
print(f"Training Set Size: {trainingSetSize}")

Training Set Size: 6000


In [17]:
# One-Hot Encoding
xenc = F.one_hot(inputs, num_classes=NUM_CLASSES).float()

# Building the NN -> (Weight Matrix)
W = torch.randn((NUM_CLASSES, 27), generator=g, requires_grad=True)

In [48]:
stepSize = 100
for iterations in range(100):
  # Forward-Pass

  regularization = 0
  logits = W[inputs, :] # logits -> log-counts

  # Matrix Multiplication 
  # logits = xenc @ W 
  counts = logits.exp() # getting Counts

  # Normalizing -> converting counts into a Probability Distribution
  P = counts / counts.sum(1, keepdims=True)

  # Finding Loss -> product of probablities of observing the data, converted into negative logs 
  nlls = torch.zeros(trainingSetSize)
  for result_i in range(trainingSetSize):
#     x, y = inputs[i].item(), labels[i].items()
    
    label_i = labels[result_i].item()
    likelihood = P[result_i, label_i]
    nll = -torch.log(likelihood)

    nlls[result_i] = nll

  loss = nlls.mean() + regularization

  # Backward-Pass
  W.grad = None
  loss.backward()

  # Gradient Descent
  W.data -= stepSize * W.grad

  print(f"negative log likelihood (loss): {loss.item()}")

negative log likelihood (loss): 1.5692129135131836
negative log likelihood (loss): 1.5691012144088745
negative log likelihood (loss): 1.5689901113510132
negative log likelihood (loss): 1.5688796043395996
negative log likelihood (loss): 1.5687696933746338
negative log likelihood (loss): 1.568660855293274
negative log likelihood (loss): 1.5685522556304932
negative log likelihood (loss): 1.5684444904327393
negative log likelihood (loss): 1.5683374404907227
negative log likelihood (loss): 1.5682311058044434
negative log likelihood (loss): 1.5681251287460327
negative log likelihood (loss): 1.568019986152649
negative log likelihood (loss): 1.5679155588150024
negative log likelihood (loss): 1.5678114891052246
negative log likelihood (loss): 1.5677083730697632
negative log likelihood (loss): 1.5676056146621704
negative log likelihood (loss): 1.567503571510315
negative log likelihood (loss): 1.5674021244049072
negative log likelihood (loss): 1.5673013925552368
negative log likelihood (loss): 1.

In [23]:
for i in range(20):
  out = []
  ix1 = None
  ix2 = 0

  while True:

    if ix1 == None:
      ix = ix2
    else:
      ix = stoi[itos[ix1] + itos[ix2]]

    logits = W[ix].clone().detach()
    count = logits.exp()
    probDist = count / count.sum()
    ix3 = torch.multinomial(probDist, num_samples=1, replacement=True, generator=g).item()
    
#     if ix1 != None: print(f"<{itos[ix1]}{itos[ix2]}> - {itos[ix3]}")
#     else: print(f"<{itos[ix2]}> - {itos[ix3]}")

    if ix3 == 0:
      break
    out.append(itos[ix3])
    ix1, ix2 = ix2, ix3
    
  print(''.join(out))




randrie
wila
phitson
oan
on
paisaislortumlsuppvmvqde
ymaraxqcssielanah
gainnia

oana
marlee
oanina
olqfdwhoajzccdlee
zacie
gani
zsailla
za
zophhocpdgo
patalee
zakcdkf
